In [ ]:
from dash import Dash, dcc, html, Input, Output, callback, dash_table
import PythonMeta as PMA
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.tools import mpl_to_plotly
import plotly.graph_objs as go

df = pd.read_excel(r'C:\Users\delil\D1\TestingPandas.xlsx',dtype=str)

#initialize
Metabolite =[
    "Saito2016,1.1151,0.1755,30,0.8613,0.1553,30",
    "Johnson2018,91.42,91.44,29,61.92,40.71,14","Johnson2019,92,94,59,43,21,12"]

#initialize
SelectedComparison = '70 year old vs 30 year old Blood alanine'


app = Dash(__name__)

def correctmetabolite(metabolite):
    return df.loc[df['comparison']==metabolite].reset_index()

def Data(metabolite):
    lists = []
    a = correctmetabolite(metabolite)
    for i in range(1,len(a)):
        b = a.iloc[i].to_numpy()
        c = b[3:]
        lists.append(','.join(c))
    return lists

def Settings(SelectedComparison):
    compare1 = correctmetabolite(SelectedComparison).loc[0][4]
    compare2 = correctmetabolite(SelectedComparison).loc[0][5]
    MetaboliteName = correctmetabolite(SelectedComparison).loc[0][2]
    Units = 'Units: ' + correctmetabolite(SelectedComparison).loc[0][3]
    Organ = "Organ: " + correctmetabolite(SelectedComparison).loc[0][6]

          
    if len(Data(SelectedComparison)) > 2:
        Model = 'Random'
    else:
        Model = 'Fixed'     
    
    if len(Units) > 2:
        Effect = 'SMD'
    else:
        Effect = 'MD'
        
    return [compare1,compare2,MetaboliteName,Units,Organ,Model,Effect]



def showstudies(studies,dtype,SelectedComparison):
    compare1 = Settings(SelectedComparison)[0]
    compare2 = Settings(SelectedComparison)[1]
    MetaboliteName = Settings(SelectedComparison)[2]
    Units = Settings(SelectedComparison)[3]
    Organ = Settings(SelectedComparison)[4]
    Model = Settings(SelectedComparison)[5]
    Effect = Settings(SelectedComparison)[6]
    #show continuous data
    if dtype.upper()=="CONT":
        allrows = []
        #this pulls out each value, study ID/SD/Mean and put them in their own column
            
        for i in range(len(studies)):
            #allrows.append(new_list)
            allrows.append([
            str(studies[i][6]),        #study ID
            str(studies[i][0]),   #mean of group1
            str(studies[i][1]),   #SD of group1
            str(studies[i][2]),   #total num of group1
            str(studies[i][3]),   #mean of group2
            str(studies[i][4]),   #SD of group2
            str(studies[i][5]) ])   #total num of group2
            
        # this separates the values in the column so that it is a list which can be passed into pandas without changing the format
        studyID = []
        Mean = []
        SD = []
        N = []
        Mean2 = []
        Sd2 = []
        N2 = []
        for i in range(len(allrows)):
            studyID.append(allrows[i][0])
            Mean.append(allrows[i][1])
            SD.append(allrows[i][2])
            N.append(allrows[i][3])
            Mean2.append(allrows[i][4])
            Sd2.append(allrows[i][5])
            N2.append(allrows[i][6])
            

        # this makes a table with only Study ID as the actual title this makes there be no columns
        # the columns are '' ' ' '       ' because the spacing has to be different
        # pandas will think that the columns are the same and stack them if the spacing is the same
        data = {MetaboliteName: [""],Units: [''],Organ: [compare1], '      ':[''],'  ': [' '], '   ':[' '], '    ': [compare2], '     ': ['']}
        
        # makes the dataframe
        df = pd.DataFrame(data)
        # df.loc just adds a row below the data table that is being used as the actual columns names
        df.loc[len(df)] = ['StudyID','Mean', "Standard Deviation", "Sample Size",'----', 'Mean', "Standard Deviation", "Sample Size"]
        # makes a datatable with the actual values being used in the table
        df2data = {MetaboliteName: studyID,Units: Mean,Organ: SD, '      ':N,'  ': '--',
                    '   ':Mean2, '    ': Sd2, '     ': N2}
        df2 = pd.DataFrame(df2data)
        # stack the tables on top of eachother both tables have to have the same column names
        df = pd.concat([df,df2])
        # add a space between this table and the next table
        df.loc[len(df)] = ['','','','','','','','']
        # hides the index       
        df.style.hide_index()
    #return df
    return [studyID,df]



def showresults(rults,SelectedComparison):
    compare1 = Settings(SelectedComparison)[0]
    compare2 = Settings(SelectedComparison)[1]
    MetaboliteName = Settings(SelectedComparison)[2]
    Units = Settings(SelectedComparison)[3]
    Organ = Settings(SelectedComparison)[4]
    Model = Settings(SelectedComparison)[5]
    Effect = Settings(SelectedComparison)[6]
    # text 1 is the titles being used
    text1 = "Study ID  N  EffectSize [95%CI]  Weight(%)" 
    StudyID = []
    TotalN = []
    EffectSize = []
    Weight = []
    AllN = []
    totaleffect = []
    LowerConIn = []
    HigherConIn = []
    # this pulls out all of the values as a single string which has to be split below
    for i in range(1,len(rults)):

        StudyID.append(rults[i][0])     #study ID
        TotalN.append(rults[i][5])     #total num
        EF = (str(np.round(rults[i][1],3) ) )   #effect size
        lowerCI = (str(np.round(rults[i][3],3)))     #lower of CI
        LowerConIn.append(np.round(rults[i][3],3))
        higherCI = (str(np.round(rults[i][4],3)) )   #higher of CI 
        EffectSize.append(EF+ "[" + lowerCI + ","+higherCI + ']')                                  
        Weight.append(round(100*(rults[i][2]/rults[0][2]), 2) )#weight
        HigherConIn.append(np.round(rults[i][4],3))

        AllEffect = rults[0][0]     #total effect size name
        AllN = np.round(rults[0][5],3)     #total N (all studies)
        TotalEF = (str(np.round(rults[0][1],3) ) )#total effect size
        LowerCI = (str(np.round(rults[0][3],3))) #total lower CI
        HigherCI = (str(np.round(rults[0][4],3)) )#total higher CI
        totaleffect = (TotalEF+ "[" + LowerCI + ","+ HigherCI + ']')
    weight = np.round(np.sum(Weight),3)


    text3 = "%d studies included  (N=%d)"%(len(rults)-1,rults[0][5])
    text4 = "Heterogeneity:Tau\u00b2=%.3f "%(rults[0][12]) if not rults[0][12]==None else "Heterogeneity:  "
    text4 += " Q(Chisquare)=%.2f  (p=%s);  I\u00b2=%s"%(
        rults[0][7],     #Q test value
        rults[0][8],     #p value for Q test
        str(round(rults[0][9],2))+"%")   #I-square value
    text6 = "Overall effect test:  z=%.2f, p=%s"%(rults[0][10],rults[0][11])  #z-test value and p-value
    
    # makes the tables blank tables with the same columns names as the last table
    data = {MetaboliteName: [""],Units: [''],Organ: [''], '      ':[''],'  ': [' '], 
            '   ':[' '], '    ': [''], '     ': ['']}
    df = pd.DataFrame(data)
    # the actual data table values
    data2 = {MetaboliteName: StudyID,Units: TotalN,Organ: EffectSize, '      ':Weight,'  ': ' ', 
            '   ':' ', '    ': '', '     ': ''}
    
    df2 = pd.DataFrame(data2)
    
    # splits the values so that they are in lists and no a single string
    text1 = text1.split('  ')
    text3 = text3.split('  ')
    text4 = text4.split('  ')
    text6 = text6.split('  ')
    
    # this has all the meta analysis calculations printed
    df.loc[len(df)] = [text1[0],text1[1], text1[2], text1[3],'', "", "", '']
    df = pd.concat([df,df2]) 
    df.loc[len(df)] = ['Total',AllN, totaleffect, weight, '', "", "", '']
    df.loc[len(df)] = [text3[0],text3[1], '', '', '', "", "", '']
    df.loc[len(df)] = [text4[0],text4[1], text4[2], text4[3],'', "", "", '']
    df.loc[len(df)] = [text6[0],text6[1], '', '','', '', "", ""]
    

    return [df,Weight,LowerConIn,HigherConIn]

def thesettings(SelectedComparison):
    Model = Settings(SelectedComparison)[5]
    Effect = Settings(SelectedComparison)[6]
    
    settings={
    "datatype":"CONT",  #for CONTinuous data
    "models":Model,             #models: Fixed or Random
    "algorithm":"IV",             #algorithm: IV
    "effect":Effect}
    return settings
 #   main(samp_cont,settings)






def main(stys,settings, SelectedComparison):
    settings = thesettings(SelectedComparison)
    compare1 = Settings(SelectedComparison)[0]
    compare2 = Settings(SelectedComparison)[1]
    MetaboliteName = Settings(SelectedComparison)[2]
    Units = Settings(SelectedComparison)[3]
    Organ = Settings(SelectedComparison)[4]
    Model = Settings(SelectedComparison)[5]
    Effect = Settings(SelectedComparison)[6]
    
    d = PMA.Data()  #Load Data class
    m = PMA.Meta()  #Load Meta class
    f = PMA.Fig()   #Load Fig class
    
    #You should always tell the datatype first!!!
    d.datatype = settings["datatype"]                #set data type, 'CATE' for binary data or 'CONT' for continuous data
    studies = d.getdata(stys)                        #load data
    #studies = d.getdata(d.readfile("studies.txt"))  #get data from a data file, see examples of data files
    showstudies(studies,d.datatype,SelectedComparison)       #show studies
#    print(showstudies(studies,d.datatype))
    m.subgroup=d.subgroup
    # make data table titles change
    #f.title = '20 Year Olds vs 60 Year Olds'
    #f.size = [6,6]                #set the subgroup
    m.datatype=d.datatype                            #set data type for meta-analysis calculating
    m.models = settings["models"]                    #set effect models: 'Fixed' or 'Random'
    m.algorithm = settings["algorithm"]              #set algorithm, based on datatype and effect size
    m.effect = settings["effect"]                    #set effect size:RR/OR/RD for binary data; SMD/MD for continuous data
    results = m.meta(studies)                        #performing the analysis

    #    print(m.models + " " + m.algorithm + " " + m.effect)
                     #show results table
    #fig = f.forest(results)
    # saves the fig to jupyter not sure how to get it out
#    fig.savefig('cystine.pdf')#show forest plot
    # this creates the plot as a matplotlib.figure.Figure
    
    DisplayStudies = showstudies(studies,d.datatype,SelectedComparison)[1]
    studyID = showstudies(studies,d.datatype,SelectedComparison)[0]
    # just shows the SMD FIXED IV so the EF type models type and variance calculations
    DisplayStudies.loc[len(DisplayStudies)] = ["Effect measure: "+str(settings["effect"]),"Effect model: "+(str(settings["models"])),"Algorithm: "+str(settings["algorithm"]),'','','','','']
    MetaAnalysisResults = showresults(results,SelectedComparison)[0]
    weight = showresults(results,SelectedComparison)[1]
    lowerCI = showresults(results,SelectedComparison)[2]
    HigherCI = showresults(results,SelectedComparison)[3]
    df = pd.concat([DisplayStudies,MetaAnalysisResults])
    
    def sigfigs(x):
        try:
            return "{:.2f}".format(float(x))
        except:
            return x  


    DataTable = df.applymap(sigfigs).style.hide_index()
    
    # return a gives the datatable
    # type(f) = PythonMeta.core.Fig
    # this turns the core.fig into a matplotlib figure that can be change
    # fig is now the most updated figure will all the information that was added such as titles
    fig = plt.figure(f.forest(results))
    
    # define what is left and what is right

    plt.xlabel("{:<38}{:<s}".format(compare1,compare2),fontsize=17)
    # define titles

    plt.title("Comparing " +compare1 + " and " + compare2,loc="center")
     
    #plt.show()
    
    return [studyID,fig,DataTable, weight,lowerCI,HigherCI]

        
'''if __name__ == '__main__':
    
    samp_cont= Metabolite
    Model = Settings(SelectedComparison)[5]
    Effect = Settings(SelectedComparison)[6]
    
    #sample 2: continuous data
    settings={
    "datatype":"CONT",  #for CONTinuous data
    "models":Model,             #models: Fixed or Random
    "algorithm":"IV",             #algorithm: IV
    "effect":Effect}                #effect size: MD, SMD
 #   main(samp_cont,settings)'''

def PlotlyFigure(themetabolite,SelectedComparison):
    settings = thesettings(SelectedComparison)
    results = main(themetabolite,settings,SelectedComparison)
    meta = results[2].data
    labels = results[0]+["Overall"]

    tickvals = []

    LowerCI = results[4]
    HigherCI = results[5]
    midpoint = np.add(HigherCI,LowerCI)/2
    for i in range(len(labels)):
        tickvals.append(len(labels)+2-i)

    tickvals[-1] = 2

    study_weights = results[3]


    plotly_figure = mpl_to_plotly(results[1])

    hover_template = "%{x}"
    plotly_figure.update_traces(hovertemplate=hover_template, selector=dict(type="scatter"),name='Confidence Interval', hoverlabel=dict(font=dict(size=16)))


# Add markers to specific points on the lines
    plotly_figure.add_trace(go.Scatter(x=midpoint, y=tickvals, hovertemplate='%{marker.size}%',
                                   mode='markers', marker=dict(color='blue', size=study_weights, symbol="square"),name='Weight'))

    plotly_figure.update_traces(hoverlabel=dict(font=dict(size=16)))

    return [meta,plotly_figure.update_layout(
    width=630, 
    height=680,  title_font_size=19, title_x=0.55,yaxis=dict(tickvals=tickvals, ticktext=labels, tickfont=dict(size=14)),
        xaxis=dict(
        tickfont=dict(size=14)))] # Adjust the left margin to center the graph horizontally
    

meta = PlotlyFigure(Metabolite,SelectedComparison)[0]


app.layout = html.Div([
    dcc.Dropdown(df.comparison.unique(), id='dropdown-menu', value=df.comparison.unique()[0]),
    html.Div(id = 'table', style={'margin-top': '1%'}),
    html.Div(dcc.Graph(id='plot'),style={'margin-left': '23%','margin-top': '3%'}),
    html.Div(id='output-container'),
    html.Div(id='output-container-2'), html.Div(id='output-container-b') # Add a placeholder component
])



@app.callback(
    Output('plot', 'figure'),
    [Input('dropdown-menu', 'value')]
)
def update_plot(selected_metabolite):
    # Call the PlotlyFigure function with the selected metabolite
    thedata = Data(selected_metabolite)
    return PlotlyFigure(thedata,selected_metabolite)[1]
    # Ensure that the function returns a valid Plotly figure object




@app.callback(
    Output('table', 'children'),
    [Input('dropdown-menu', 'value')]
)
def update_variable_a(selected_metabolite):
    # Call the PlotlyFigure function with the selected metabolite
    thedata = Data(selected_metabolite)
    meta =  PlotlyFigure(thedata,selected_metabolite)[0]
    return dash_table.DataTable(meta.to_dict('records'),[{'name': i, 'id': i} for i in meta],
                export_format="csv",
                style_cell={'textAlign': 'left','font-family':'Open Sans','background-color': '#F2F2F2'})
    # Ensure that the function returns a valid Plotly figure object








if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, jupyter_mode="external") 


